In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error

In [2]:
all_properties = pd.read_csv("../data/curated/houses_all_properties.csv")
all_properties.head()

,address,parking,type,num_schools,cost,suburb,beds,baths,cost/(beds+baths),geometry,...,NUMBER_OF_JOBS_PERSONS_2018-19,NUMBER_OF_JOBS_PERSONS_2019-20,NUMBER_OF_JOBS_PERSONS_2020-21,MEDIAN_INCOME_PERSONS_2016-17,MEDIAN_INCOME_PERSONS_2017-18,MEDIAN_INCOME_PERSONS_2018-19,MEDIAN_INCOME_PERSONS_2019-20,MEDIAN_INCOME_PERSONS_2020-21,centroid,distance_to_centroid
0,"(Leased) 3 Yarra Street, South Yarra VIC 3141",1,Apartment / Unit / Flat,13,460.0,South Yarra,1,1,230.00,POINT (144.992828 -37.838173),...,13060.0,12759.0,12394.0,26306.5,27750.5,29043.5,32390.5,34577.5,POINT (144.9965000772275 -37.83614617221807),0.004194
1,"004B/12 Albert Street, Hawthorn East VIC 3123",0,Apartment / Unit / Flat,14,400.0,Hawthorn East,1,1,200.00,POINT (145.048408 -37.823472),...,15422.0,14730.0,14159.0,26100.5,27117.0,26803.5,31088.0,34211.0,POINT (145.04991149745675 -37.83126983822268),0.007941
2,"04/949 Dandenong Road, Malvern East VIC 3145",0,Apartment / Unit / Flat,10,365.0,Malvern East,1,1,182.50,POINT (145.048262 -37.878631),...,18950.0,18895.0,19034.0,25636.0,26307.0,25956.5,28608.0,30642.5,POINT (145.065853863916 -37.87682309041124),0.017685
3,"0711/333 Exhibition St, Melbourne VIC 3000",0,Apartment / Unit / Flat,12,675.0,Melbourne,2,2,168.75,POINT (144.96834 -37.807831),...,13693.0,12473.0,11002.0,10656.0,10929.0,11120.5,12060.0,14478.0,POINT (144.96859287539095 -37.812787018836396),0.004962
4,"1 & 2/23 Koonawarra Street, Clayton VIC 3168",1,House,5,340.0,Clayton,1,1,170.00,POINT (145.124589 -37.90863),...,6188.0,6445.0,7536.0,11981.0,12020.0,11625.0,12689.5,12635.0,POINT (145.1381453634467 -37.91053619756948),0.013690


In [3]:
stratify_column = 'SA2_Name'

house_counts = all_properties[stratify_column].value_counts()

all_properties = all_properties[all_properties[stratify_column].isin(house_counts[house_counts >= 50].index)]

In [4]:
# Create a mapping dictionary
mapping = {category: index for index, category in enumerate(all_properties['SA2_Name'].unique())}

# Map the categorical variable to numerical indices
all_properties['map_SA2_Name'] = all_properties['SA2_Name'].map(mapping)

all_properties['type'] = all_properties['type'].apply(lambda x: 0 if x == 'House' else 1) 

In [5]:
columns_to_drop = []
    
for item in all_properties.columns:
    if "name" in item:
        columns_to_drop.append(item)
    if "centroid" in item:
        columns_to_drop.append(item)
    if "increase" in item:
        columns_to_drop.append(item)
    if "migration" in item:
        columns_to_drop.append(item)
    if "ERP" in item:
        columns_to_drop.append(item)
    if "NUMBER" in item:
        columns_to_drop.append(item)
    if "MEDIAN" in item:
        columns_to_drop.append(item)
        
all_properties = all_properties.drop(columns_to_drop, axis = 1)

all_properties = all_properties.drop(["address", "cost/(beds+baths)", "suburb", "geometry"], axis = 1)

all_properties.dtypes

parking                                int64
type                                   int64
num_schools                            int64
cost                                 float64
beds                                   int64
baths                                  int64
closest_train_station_distance_km    float64
closest_tram_station_distance_km     float64
closest_hospital_distance_km         float64
closest_grocery_distance_km          float64
SA2_Name                              object
map_SA2_Name                           int64
dtype: object

In [6]:
exclude_columns = ['parking', 'type', 'num_schools', 'beds', 'baths', 'map_SA2_Name', 'cost']
columns_to_scale = [col for col in all_properties.columns if all_properties[col].dtype != object and col not in exclude_columns]

# Initialize the StandardScaler
scaler = MinMaxScaler()

# Scale the selected columns
data_scaled = all_properties.copy()
data_scaled[columns_to_scale] = scaler.fit_transform(data_scaled[columns_to_scale])

In [7]:
display(data_scaled.describe())

,parking,type,num_schools,cost,beds,baths,closest_train_station_distance_km,closest_tram_station_distance_km,closest_hospital_distance_km,closest_grocery_distance_km,map_SA2_Name
count,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000,4430.000000
mean,1.017833,0.640858,8.681264,642.020377,2.358239,1.498871,0.062190,0.068235,0.087257,0.068030,24.918962
std,0.890175,0.479803,3.587438,260.052483,1.101988,0.564530,0.136905,0.141491,0.152086,0.147144,14.836719
min,0.000000,0.000000,0.000000,165.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,7.000000,500.000000,2.000000,1.000000,0.010226,0.000433,0.019542,0.010065,12.000000
50%,1.000000,1.000000,10.000000,580.000000,2.000000,1.000000,0.021734,0.000979,0.043614,0.021595,24.000000
75%,2.000000,1.000000,11.000000,700.000000,3.000000,2.000000,0.042771,0.080316,0.080021,0.053400,39.000000
max,11.000000,1.000000,15.000000,3800.000000,9.000000,6.000000,1.000000,1.000000,1.000000,1.000000,46.000000


In [8]:
X = data_scaled.drop(columns=["cost", "SA2_Name"])
y = data_scaled["cost"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=data_scaled['map_SA2_Name']
)

In [9]:
linear_model = LinearRegression()

linear_model.fit(X_train, y_train)

y_pred_linear_test = linear_model.predict(X_test)
y_pred_linear_train = linear_model.predict(X_train)

rmse_linear_test = root_mean_squared_error(y_test, y_pred_linear_test)
rmse_linear_train = root_mean_squared_error(y_train, y_pred_linear_train)

mae_linear_test = mean_absolute_error(y_test, y_pred_linear_test)
mae_linear_train = mean_absolute_error(y_train, y_pred_linear_train)

r2_linear_test = r2_score(y_test, y_pred_linear_test) 
r2_linear_train = r2_score(y_train, y_pred_linear_train) 

print(f'Root Mean Squared Error: Test: {rmse_linear_test} and Train: {rmse_linear_train}')
print(f'Mean Absolute Error: Test: {mae_linear_test} and Train: {mae_linear_train}')
print(f'R² Score: Test: {r2_linear_test} and Train: {r2_linear_train}')

Root Mean Squared Error: Test: 198.22322590635727 and Train: 183.77903964289487
Mean Absolute Error: Test: 131.10484603852544 and Train: 121.04589679709893
R² Score: Test: 0.4687784904811736 and Train: 0.4884027053796246


In [29]:
coefficients = linear_model.coef_

feature_names = X_train.columns if hasattr(X_train, 'columns') else range(X_train.shape[1])
coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Sort the DataFrame by coefficients
coefficients_df = coefficients_df.sort_values(by='Coefficient', ascending=False)

print(coefficients_df)

                             Feature  Coefficient
5  closest_train_station_distance_km   728.905679
4                              baths   195.649057
3                               beds   115.776067
1                               type    75.876570
2                        num_schools    42.231165
0                            parking     9.552825
9                       map_SA2_Name     2.266714
8        closest_grocery_distance_km  -174.969029
7       closest_hospital_distance_km  -283.531185
6   closest_tram_station_distance_km  -425.981565


In [10]:
rf_model = RandomForestRegressor(max_depth = 5)

rf_model.fit(X_train, y_train)

y_pred_rf_test = rf_model.predict(X_test)
y_pred_rf_train = rf_model.predict(X_train)

rmse_rf_test = root_mean_squared_error(y_test, y_pred_rf_test) 
rmse_rf_train = root_mean_squared_error(y_train, y_pred_rf_train)

mae_rf_test = mean_absolute_error(y_test, y_pred_rf_test)
mae_rf_train = mean_absolute_error(y_train, y_pred_rf_train)

r2_rf_test = r2_score(y_test, y_pred_rf_test)
r2_rf_train = r2_score(y_train, y_pred_rf_train)

print(f'Root Mean Squared Error: Test: {rmse_rf_test} and Train: {rmse_rf_train}')
print(f'Mean Absolute Error: Test: {mae_rf_test} and Train: {mae_rf_train}')
print(f'R² Score: Test: {r2_rf_test} and Train: {r2_rf_train}')

Root Mean Squared Error: Test: 151.16959850415586 and Train: 130.89252722467683
Mean Absolute Error: Test: 89.52408852921242 and Train: 83.60146200419616
R² Score: Test: 0.6910448125063056 and Train: 0.7404828938984682


In [27]:
importance = rf_model.feature_importances_

feature_names = X_train.columns if hasattr(X_train, 'columns') else range(X_train.shape[1])
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance})

# Sort the DataFrame by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

                             Feature  Importance
6   closest_tram_station_distance_km    0.344375
3                               beds    0.276806
4                              baths    0.233299
0                            parking    0.053222
5  closest_train_station_distance_km    0.042953
9                       map_SA2_Name    0.013784
7       closest_hospital_distance_km    0.012552
8        closest_grocery_distance_km    0.012148
2                        num_schools    0.009868
1                               type    0.000994


In [11]:
y_pred_series = pd.Series(rf_model.predict(X), name='Predicted').reset_index(drop=True)
X_reset = X.reset_index(drop=True)
predicted_test = pd.concat([X_reset, y_pred_series], axis=1)
predicted_test = pd.concat([predicted_test, data_scaled["SA2_Name"]], axis = 1)

In [12]:
aggregated_df = predicted_test.groupby("SA2_Name")['Predicted'].median().reset_index(name='SA2_Median')
aggregated_df

,SA2_Name,SA2_Median
0,Albert Park,548.824624
1,Alfredton,544.452904
2,Barwon Heads - Armstrong Creek,628.939063
3,Bendigo,543.806129
4,Carlton,544.657806
5,Carnegie,617.434256
6,Caulfield - North,612.946058
7,Clyde North - South,544.934680
8,Docklands,544.934680
9,Doncaster,617.434256


In [13]:
threshold_subs = aggregated_df["SA2_Name"].unique()

In [14]:
len(threshold_subs)

47

In [15]:
subs_w_hist = pd.read_csv("../data/curated/suburb_w_hist.csv")
subs_w_hist['Suburb'] = subs_w_hist['Suburb'].replace('Melbourne', 'CBD')

In [16]:
contain = []
not_contain = []

for suburb in subs_w_hist["Suburb"].unique():
    found = False
    for sub in threshold_subs:
        if suburb == sub:
            contain.append((suburb, sub))
            break
        elif suburb in sub:
            contain.append((suburb, sub))
            found = True
    if not found:
        not_contain.append(suburb)
    
comp = pd.DataFrame(contain, columns = ["sub_w_hist", "sa2"])

In [17]:
len(comp["sa2"].unique())

35

In [18]:
from collections import Counter

count = Counter(comp["sa2"])

duplicate = []

for item in count:
    if count[item] > 1:
        print(item, count[item])
        duplicate.append(item)

Malvern - Glen Iris 2
Hawthorn East 2
Malvern East 2
Prahran - Windsor 2
St Kilda East 2


In [20]:
results = []

for suburb in comp["sa2"]:
    
    if suburb in duplicate:
        if "-" in suburb:
            hist_subs = list(comp[comp["sa2"] == suburb]["sub_w_hist"])
            hist_data = pd.concat([subs_w_hist[subs_w_hist["Suburb"] == hist_subs[0]][['date', 'median_rental_price']], 
                                   subs_w_hist[subs_w_hist["Suburb"] == hist_subs[1]][['date', 'median_rental_price']]])
        else:
            hist_subs = list(comp[comp["sa2"] == suburb]["sub_w_hist"])
            if hist_subs[0] == suburb:
                hist_data = subs_w_hist[subs_w_hist["Suburb"] == hist_subs[0]][['date', 'median_rental_price']]
            else:
                hist_data = subs_w_hist[subs_w_hist["Suburb"] == hist_subs[1]][['date', 'median_rental_price']]
                
    else:
        hist_sub = list(comp[comp["sa2"] == suburb]["sub_w_hist"])[0]
        hist_data = subs_w_hist[subs_w_hist["Suburb"] == hist_sub][['date', 'median_rental_price']]
        
    sep_2024_median_df = pd.DataFrame({'date': '2024-09-01', 'median_rental_price': aggregated_df[aggregated_df["SA2_Name"] == suburb]["SA2_Median"]})
    suburb_df = pd.concat([hist_data, sep_2024_median_df], ignore_index=True)
    suburb_df['date'] = pd.to_datetime(suburb_df['date'])
    
    # Use only recent data for linear regression
    suburb_df = suburb_df[suburb_df['date'] >= '2021-01-01']

    # Extract the year from the date
    suburb_df['year'] = suburb_df['date'].dt.year

    # Prepare data for the linear regression model
    X_historical = suburb_df[['year']]  # Independent variable (year)
    y_historical = suburb_df['median_rental_price']  # Dependent variable (rental price)

    # Fit a linear regression model
    future_model = LinearRegression()
    future_model.fit(X_historical, y_historical)

    # Extract the coefficient (slope), which represents the growth rate
    growth_rate = future_model.coef_[0]

    # Forecast for the years 2025, 2026, and 2027
    forecast_years = pd.DataFrame({'year': [2025, 2026, 2027]})
    forecasted_prices = future_model.predict(forecast_years)

    # Store the suburb's forecasted prices and growth rate in one row
    result_row = {
        'Suburb': suburb,
        '2025': forecasted_prices[0],
        '2026': forecasted_prices[1],
        '2027': forecasted_prices[2],
        'growth_rate': growth_rate
    }
    
    results.append(result_row)

# Create a DataFrame from the results
final_df = pd.DataFrame(results)

# Reorder the DataFrame columns to have 'Suburb' first, followed by the years and growth rate
final_df = final_df[['Suburb', '2025', '2026', '2027', 'growth_rate']]

In [21]:
results = []

for suburb in comp["sa2"]:
    
    if suburb in duplicate:
        if "-" in suburb:
            hist_subs = list(comp[comp["sa2"] == suburb]["sub_w_hist"])
            hist_data = pd.concat([subs_w_hist[subs_w_hist["Suburb"] == hist_subs[0]][['date', 'median_rental_price']], 
                                   subs_w_hist[subs_w_hist["Suburb"] == hist_subs[1]][['date', 'median_rental_price']]])
        else:
            hist_subs = list(comp[comp["sa2"] == suburb]["sub_w_hist"])
            if hist_subs[0] == suburb:
                hist_data = subs_w_hist[subs_w_hist["Suburb"] == hist_subs[0]][['date', 'median_rental_price']]
            else:
                hist_data = subs_w_hist[subs_w_hist["Suburb"] == hist_subs[1]][['date', 'median_rental_price']]
                
    else:
        hist_sub = list(comp[comp["sa2"] == suburb]["sub_w_hist"])[0]
        hist_data = subs_w_hist[subs_w_hist["Suburb"] == hist_sub][['date', 'median_rental_price']]
        
    sep_2024_median_df = pd.DataFrame({'date': '2024-09-01', 'median_rental_price': aggregated_df[aggregated_df["SA2_Name"] == suburb]["SA2_Median"]})
    suburb_df = pd.concat([hist_data, sep_2024_median_df], ignore_index=True)
    suburb_df['date'] = pd.to_datetime(suburb_df['date'])
    
    suburb_df['median_rental_price'] = pd.to_numeric(suburb_df['median_rental_price'], errors='coerce')
    
    # Use only recent data for linear regression
    suburb_df = suburb_df[suburb_df['date'] >= '2021-01-01']

    # Extract the year from the date
    suburb_df['year'] = suburb_df['date'].dt.year

    # Prepare data for the linear regression model
    X_historical = suburb_df[['year']]  # Independent variable (year)
    y_historical = suburb_df['median_rental_price']  # Dependent variable (rental price)

    # Fit a linear regression model
    future_model = LinearRegression()
    future_model.fit(X_historical, y_historical)

    # Extract the coefficient (slope), which represents the yearly growth rate
    growth_rate = future_model.coef_[0]

    # Forecast for the years 2025, 2026, and 2027
    forecast_years = pd.DataFrame({'year': [2025, 2026, 2027]})
    forecasted_prices = future_model.predict(forecast_years)

    # Calculate growth rate percentage for each year
    price_2024 = float(suburb_df[suburb_df['date'].dt.year == 2024]["median_rental_price"].median())

    growth_2025 = (forecasted_prices[0] - price_2024) / price_2024 * 100
    growth_2026 = (forecasted_prices[1] - forecasted_prices[0]) / forecasted_prices[0] * 100
    growth_2027 = (forecasted_prices[2] - forecasted_prices[1]) / forecasted_prices[1] * 100

    # Store the suburb's forecasted prices and growth rate in one row
    result_row = {
        'Suburb': suburb,
        '2025': forecasted_prices[0],
        '2026': forecasted_prices[1],
        '2027': forecasted_prices[2],
        'growth_2025': growth_2025,
        'growth_2026': growth_2026,
        'growth_2027': growth_2027
    }
    
    results.append(result_row)

# Create a DataFrame from the results
final_df = pd.DataFrame(results)

final_df = final_df[['Suburb', '2025', '2026', '2027', 'growth_2025', 'growth_2026', 'growth_2027']]

for col in ["growth_2025", "growth_2026", "growth_2027"]:
    final_df[col] = final_df[col].apply(lambda x: float(x.iloc[0]) if isinstance(x, pd.Series) else x)

In [22]:
final_df.to_csv("../data/curated/future_predict.csv")